In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
#don't rerun this
# !git clone https://github.com/PediaMedAI/PIE.git /content/drive/MyDrive/GenAIProject/PIE/

fatal: destination path '/content/drive/MyDrive/GenAIProject/PIE' already exists and is not an empty directory.


In [ ]:
#don't rerun this
#!git clone https://github.com/ysuter/gbm-data-longitudinal.git /content/drive/MyDrive/GenAIProject/PIE/data/

fatal: destination path '/content/drive/MyDrive/GenAIProject/PIE/data' already exists and is not an empty directory.


In [ ]:
%cd /content/drive/MyDrive/GenAIProject/
#!pip install -r requirements.txt

/content/drive/MyDrive/GenAIProject


In [ ]:
#convert from 3d MRI to 2d
#don't rerun
import os
import nibabel as nib
import numpy as np
import cv2
from PIL import Image
data = "/content/drive/MyDrive/GenAIProject/PIE/data/gbm/cleanimages"
finaloutput = "/content/drive/MyDrive/GenAIProject/PIE/data/gbm/finaloutput"
os.makedirs(finaloutput, exist_ok=True)
modes = {"FLAIR.nii.gz", "CT1.nii.gz"}

def convert_to_2d(img):
    if img.max() > 0:
      img = img/img.max()
    img = (img*255).astype(np.uint8)
    return img

for patient in os.listdir(data):
  currPatient = os.path.join(data, patient)
  if os.path.isdir(currPatient):
    for week in os.listdir(currPatient):
      currWeek = os.path.join(currPatient, week)
      if os.path.isdir(currWeek):
        for currFile in os.listdir(currWeek):
          if currFile in modes:
            new = nib.load(os.path.join(currWeek, currFile)).get_fdata()
            sliceI = new.shape[2]//2
            currSlice = new[:,:,sliceI]
            if currSlice.max() >=0.05:
              currImg = convert_to_2d(currSlice)
              currPath = os.path.join(finaloutput, f"{patient}_{week}_{currFile.replace('.nii.gz', '')}_slice.png")
              cv2.imwrite(currPath, currImg)

In [ ]:
!pip install segment-anything opencv-python-headless matplotlib torch torchvision

In [ ]:
#based on PIE paper, SAM ROI mask with smoothing
#don't rerun

import os
import numpy as np
import cv2
from PIL import Image
from scipy.ndimage import gaussian_filter
from skimage.morphology import closing, disk
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

finaloutput = "/content/drive/MyDrive/GenAIProject/PIE/data/gbm/finaloutput"
masks = "/content/drive/MyDrive/GenAIProject/PIE/data/gbm/masks"
os.makedirs(masks, exist_ok=True)

samPath = "sam_vit_b_01ec64.pth"
if not os.path.exists(samPath):
  !wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

sam = sam_model_registry["vit_b"](checkpoint=samPath)
sam.to("cuda" if torch.cuda.is_available() else "cpu")

maskGenerator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=16,
    pred_iou_thresh=0.88,
    stability_score_thresh=0.90,
    min_mask_region_area=150)

#based on PIE paper
def smoothEdges(mask):
  blurredMask = gaussian_filter(mask.astype(np.float32), sigma=4)
  binaryMask = (blurredMask>0.2).astype(np.uint8)*255
  return binaryMask

for imgFile in os.listdir(finaloutput):
  imgPath = os.path.join(finaloutput, imgFile)
  currImg = Image.open(imgPath).convert("RGB")
  npImg = np.array(currImg)
  currMasks = maskGenerator.generate(npImg)
  currMasks = sorted(currMasks, key=lambda x: x["area"], reverse=True)
  mask = currMasks[0]['segmentation'].astype(np.uint8)
  smoothedMask = smoothEdges(mask)
  cv2.imwrite(os.path.join(masks, imgFile.replace("slice.png", "mask.png")), smoothedMask)


In [ ]:
%%bash
export HF_HOME=./cache/
torchrun --nproc_per_node=1 training/train.py \
            --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4" \
            --instance_data_dir="data/gbm/finaloutput_train/images" \
            --output_dir="checkpoints" \
            --instance_prompt="A glioblastoma MRI image" \
            --resolution=512 \
            --train_batch_size=2 \
            --gradient_accumulation_steps=2 \
            --learning_rate=5e-5 \
            --lr_warmup_steps=1000 \
            --max_train_steps=5000 \
            --lr_scheduler "cosine" \
            --checkpoints_total_limit 2 \
            --gradient_checkpointing \
            --mixed_precision bf16 \
            --center_crop \
            --instance_dataset glioblastoma \
            --checkpointing_steps 5000

2025-11-23 19:17:37.430898: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-23 19:17:37.448239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763925457.469236    7550 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763925457.475630    7550 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763925457.491800    7550 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
!pip install git+https://github.com/openai/CLIP.git
!pip install ftfy regex tqdm

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-vfs80yda
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-vfs80yda
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.8 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=162a8f2db7507dc80d585893ea1563c7200e4f3871b2b711c7c73832bcc03849
  Stored in directory: /tmp/pip-ephem-wheel-cache-htl04jvo/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [ ]:
#baseline clip score
import os
import torch
import clip
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)
model.eval()

root = "/content/drive/MyDrive/GenAIProject/PIE_results_all_test"
res = []

def feature_extraction(imgPath):
  img = Image.open(imgPath).convert("RGB")
  img = preprocess(img).unsqueeze(0).to(device)
  with torch.no_grad():
    imgFeats = model.encode_image(img)
    imgFeats = imgFeats/imgFeats.norm(dim=-1, keepdim=True)
  return imgFeats.cpu().numpy()

for pat in tqdm(os.listdir(root)):
  patPath = os.path.join(root, pat)
  for week in os.listdir(patPath):
    weekPath = os.path.join(patPath, week)
    imgPaths = []
    for img in os.listdir(weekPath):
      if "mask" not in img:
        imgPaths.append(os.path.join(weekPath, img))
    imgPaths = sorted(imgPaths)

    ogFeats = feature_extraction(imgPaths[0])
    newFeats = []
    for imgPath in imgPaths[1:]:
      newFeats.append(feature_extraction(imgPath))
    newFeats = np.vstack(newFeats)
    meanNew = newFeats.mean(axis=0, keepdims=True)
    currScore = cosine_similarity(ogFeats, meanNew)
    res.append({"week":week, "clipScore":float(currScore[0][0])})

dataframe = pd.DataFrame(res)
dataframe.to_csv("/content/drive/MyDrive/GenAIProject/CLIP_I_scores.csv", index=False)
finalScore = dataframe["clipScore"].mean()
print("Final mean CLIP-I score:", finalScore)

100%|██████████| 11/11 [17:18<00:00, 94.43s/it] 

Final mean CLIP-I score: 0.963709546570268


In [ ]:
!pip install libauc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.6 MB/s eta 0:00:00


In [ ]:
#densenet classification training
#adapted from: https://github.com/PediaMedAI/PIE/blob/main/metric/confidence_model/finetune.py

from torch.nn import CrossEntropyLoss
from libauc.optimizers import Adam
from libauc.models import densenet121 as DenseNet121

import torch
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import roc_auc_score

from Glioblastoma_Dataset import Glioblastoma

def set_all_seeds(SEED):
    # REPRODUCIBILITY
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# paramaters
SEED = 123
BATCH_SIZE = 16
lr = 0.001 # using smaller learning rate is better
weight_decay = 1e-5

root = "/content/drive/MyDrive/GenAIProject/PIE/data/gbm/finaloutput"
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])
dataset = Glioblastoma(root, transform)

train_len = int(0.8 * len(dataset))
train_set, val_set = random_split(dataset, [train_len, len(dataset) - train_len])

trainloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
testloader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


# model
set_all_seeds(SEED)
model = DenseNet121(pretrained=False, last_activation=None, activations='relu', num_classes=4)
model = model.cuda()

# define loss & optimizer
loss_fn = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

best_val_auc = 0
for epoch in range(10):
  model.train()
  for idx, (train_imgs, train_labels, _) in enumerate(trainloader):
      train_data = train_imgs.cuda()
      train_labels = train_labels.long().cuda()
      y_pred = model(train_data)
      loss = loss_fn(y_pred, train_labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # validation
      if idx % 25 == 0:
        model.eval()
        with torch.no_grad():
            test_pred = []
            test_true = []
            for imgs, test_label, meta in testloader:
                test_data = imgs.cuda()
                y_pred = model(test_data)
                curr_probs = F.softmax(y_pred, dim=1)
                test_pred.append(curr_probs.cpu().numpy())
                test_true.append(test_label.cpu().numpy())

            test_true = np.concatenate(test_true)
            test_pred = np.concatenate(test_pred)
            try:
              val_auc =  roc_auc_score(test_true, test_pred, multi_class='ovr')
            except:
              val_auc = 0.0
        model.train()

        if best_val_auc < val_auc:
            best_val_auc = val_auc
            torch.save(model.state_dict(), 'gbm_classifier.pth')

        print ('Epoch=%s, BatchID=%s, Val_AUC=%.4f, lr=%.4f'%(epoch, idx, val_auc,  optimizer.param_groups[0]['lr']))

print ('Best Val_AUC is %.4f'%best_val_auc)

Epoch=0, BatchID=0, Val_AUC=0.4816, lr=0.0010
Epoch=0, BatchID=25, Val_AUC=0.5390, lr=0.0010
Epoch=1, BatchID=0, Val_AUC=0.5707, lr=0.0010
Epoch=1, BatchID=25, Val_AUC=0.5746, lr=0.0010
Epoch=2, BatchID=0, Val_AUC=0.6080, lr=0.0010
Epoch=2, BatchID=25, Val_AUC=0.5917, lr=0.0010
Epoch=3, BatchID=0, Val_AUC=0.5960, lr=0.0010
Epoch=3, BatchID=25, Val_AUC=0.5475, lr=0.0010
Epoch=4, BatchID=0, Val_AUC=0.5579, lr=0.0010
Epoch=4, BatchID=25, Val_AUC=0.5751, lr=0.0010
Epoch=5, BatchID=0, Val_AUC=0.5347, lr=0.0010
Epoch=5, BatchID=25, Val_AUC=0.5354, lr=0.0010
Epoch=6, BatchID=0, Val_AUC=0.5399, lr=0.0010
Epoch=6, BatchID=25, Val_AUC=0.5148, lr=0.0010
Epoch=7, BatchID=0, Val_AUC=0.4997, lr=0.0010
Epoch=7, BatchID=25, Val_AUC=0.5010, lr=0.0010
Epoch=8, BatchID=0, Val_AUC=0.4863, lr=0.0010
Epoch=8, BatchID=25, Val_AUC=0.4721, lr=0.0010
Epoch=9, BatchID=0, Val_AUC=0.4899, lr=0.0010
Epoch=9, BatchID=25, Val_AUC=0.4886, lr=0.0010
Best Val_AUC is 0.6080


In [ ]:
#baseline classifer confidence
import os
import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image
from torchvision import transforms
from libauc.models import densenet121 as DenseNet121

root = "/content/drive/MyDrive/GenAIProject/PIE_results_all_test"
model = DenseNet121(pretrained=False, last_activation=None, activations='relu', num_classes=4)
model.load_state_dict(torch.load("gbm_classifier.pth"))
model = model.cuda()
model.eval()
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

def confidenceScore(img_path):
  img = Image.open(img_path).convert("RGB")
  img = transform(img).unsqueeze(0).cuda()
  with torch.no_grad():
    logits = model(img)
    logitsN = F.softmax(logits, dim=1)
    res = logitsN.max().item()
  return res

confidenceScores = []
for pat in os.listdir(root):
  patPath = os.path.join(root, pat)
  for week in os.listdir(patPath):
    weekPath = os.path.join(patPath, week)
    for img in os.listdir(weekPath):
      if "mask" not in img:
        currPath = os.path.join(weekPath, img)
        currScore = confidenceScore(currPath)
        confidenceScores.append(currScore)
finalScore = float(np.mean(confidenceScores))
print("Mean confidence score:", finalScore)
import json
with open("confidence_scores.json", "w") as f:
  json.dump(confidenceScores, f)


Mean confidence score: 0.6960109885872028


In [ ]:
#full bounding box Med Sam clip score
import os
import torch
import clip
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)
model.eval()

root = "/content/drive/MyDrive/GenAIProject/PIE-MedSAM_results_all_test"
res = []

def feature_extraction(imgPath):
  img = Image.open(imgPath).convert("RGB")
  img = preprocess(img).unsqueeze(0).to(device)
  with torch.no_grad():
    imgFeats = model.encode_image(img)
    imgFeats = imgFeats/imgFeats.norm(dim=-1, keepdim=True)
  return imgFeats.cpu().numpy()

for pat in tqdm(os.listdir(root)):
  patPath = os.path.join(root, pat)
  for week in os.listdir(patPath):
    weekPath = os.path.join(patPath, week)
    imgPaths = []
    for img in os.listdir(weekPath):
      if "mask" not in img:
        imgPaths.append(os.path.join(weekPath, img))
    imgPaths = sorted(imgPaths)

    ogFeats = feature_extraction(imgPaths[0])
    newFeats = []
    for imgPath in imgPaths[1:]:
      newFeats.append(feature_extraction(imgPath))
    newFeats = np.vstack(newFeats)
    meanNew = newFeats.mean(axis=0, keepdims=True)
    currScore = cosine_similarity(ogFeats, meanNew)
    res.append({"week":week, "clipScore":float(currScore[0][0])})

dataframe = pd.DataFrame(res)
dataframe.to_csv("/content/drive/MyDrive/GenAIProject/CLIP_I_scores.csv", index=False)
finalScore = dataframe["clipScore"].mean()
print("Final mean CLIP-I score:", finalScore)

100%|████████████████████████████████████████| 890M/890M [00:04<00:00, 225MiB/s]
100%|██████████| 11/11 [25:22<00:00, 138.44s/it]


Final mean CLIP-I score: 0.9520652848539459


In [ ]:
#full bounding box Med sam classifer confidence
import os
import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image
from torchvision import transforms
from libauc.models import densenet121 as DenseNet121

root = "/content/drive/MyDrive/GenAIProject/PIE-MedSAM_results_all_test"
model = DenseNet121(pretrained=False, last_activation=None, activations='relu', num_classes=4)
model.load_state_dict(torch.load("gbm_classifier.pth"))
model = model.cuda()
model.eval()
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

def confidenceScore(img_path):
  img = Image.open(img_path).convert("RGB")
  img = transform(img).unsqueeze(0).cuda()
  with torch.no_grad():
    logits = model(img)
    logitsN = F.softmax(logits, dim=1)
    res = logitsN.max().item()
  return res

confidenceScores = []
for pat in os.listdir(root):
  patPath = os.path.join(root, pat)
  for week in os.listdir(patPath):
    weekPath = os.path.join(patPath, week)
    for img in os.listdir(weekPath):
      if "mask" not in img:
        currPath = os.path.join(weekPath, img)
        currScore = confidenceScore(currPath)
        confidenceScores.append(currScore)
finalScore = float(np.mean(confidenceScores))
print("Mean confidence score:", finalScore)
import json
with open("confidence_scores.json", "w") as f:
  json.dump(confidenceScores, f)


Mean confidence score: 0.6858833790922878


In [ ]:
#auto bounding box Med Sam clip score
import os
import torch
import clip
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)
model.eval()

root = "/content/drive/MyDrive/GenAIProject/PIE-MedSAM_results_all_test_autobox"
res = []

def feature_extraction(imgPath):
  img = Image.open(imgPath).convert("RGB")
  img = preprocess(img).unsqueeze(0).to(device)
  with torch.no_grad():
    imgFeats = model.encode_image(img)
    imgFeats = imgFeats/imgFeats.norm(dim=-1, keepdim=True)
  return imgFeats.cpu().numpy()

for pat in tqdm(os.listdir(root)):
  patPath = os.path.join(root, pat)
  for week in os.listdir(patPath):
    weekPath = os.path.join(patPath, week)
    imgPaths = []
    for img in os.listdir(weekPath):
      if "mask" not in img:
        imgPaths.append(os.path.join(weekPath, img))
    imgPaths = sorted(imgPaths)

    ogFeats = feature_extraction(imgPaths[0])
    newFeats = []
    for imgPath in imgPaths[1:]:
      newFeats.append(feature_extraction(imgPath))
    newFeats = np.vstack(newFeats)
    meanNew = newFeats.mean(axis=0, keepdims=True)
    currScore = cosine_similarity(ogFeats, meanNew)
    res.append({"week":week, "clipScore":float(currScore[0][0])})

dataframe = pd.DataFrame(res)
dataframe.to_csv("/content/drive/MyDrive/GenAIProject/CLIP_I_scores.csv", index=False)
finalScore = dataframe["clipScore"].mean()
print("Final mean CLIP-I score:", finalScore)

100%|██████████| 11/11 [26:36<00:00, 145.15s/it]

Final mean CLIP-I score: 0.9440110351660373


In [ ]:
#full bounding box Med sam classifer confidence
import os
import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image
from torchvision import transforms
from libauc.models import densenet121 as DenseNet121

root = "/content/drive/MyDrive/GenAIProject/PIE-MedSAM_results_all_test_autobox"
model = DenseNet121(pretrained=False, last_activation=None, activations='relu', num_classes=4)
model.load_state_dict(torch.load("gbm_classifier.pth"))
model = model.cuda()
model.eval()
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

def confidenceScore(img_path):
  img = Image.open(img_path).convert("RGB")
  img = transform(img).unsqueeze(0).cuda()
  with torch.no_grad():
    logits = model(img)
    logitsN = F.softmax(logits, dim=1)
    res = logitsN.max().item()
  return res

confidenceScores = []
for pat in os.listdir(root):
  patPath = os.path.join(root, pat)
  for week in os.listdir(patPath):
    weekPath = os.path.join(patPath, week)
    for img in os.listdir(weekPath):
      if "mask" not in img:
        currPath = os.path.join(weekPath, img)
        currScore = confidenceScore(currPath)
        confidenceScores.append(currScore)
finalScore = float(np.mean(confidenceScores))
print("Mean confidence score:", finalScore)
import json
with open("confidence_scores.json", "w") as f:
  json.dump(confidenceScores, f)


Mean confidence score: 0.6911445083771806
